# Biclustering lasso

Biclustering lasso is a family of methods, solving different, but related problems.

## "Sources of inspiration": 

### PCA

Classical PCA solves the following problem: We are given an input data matrix $X$ of $p$ columns and $n$ rows (e.g. rows are patients and columns are gene expressions).

$X = \begin{pmatrix} 1 && 2 && 3 \\ 4 && 5 && 6 \end{pmatrix}$.

We are looking for a vector $v$ of unit length ($||v||_2 = 1$), such that projection of the data onto this vector explains as much variance in the data as possible (equivalently, the share of sum os squares of distances, preserved by such a projection, is as large as possible, or, equivalently, the discrepancy in distances between sums of squares of distances between points is as small as possible).

Denoting the covariance matrix $\Sigma = \frac{1}{n-1} X^T X$, we get:

$\begin{cases} v^T \Sigma v \to max \\ ||v||_2 = 1 \end{cases}$

Optimal solutions of this problem correspond to eigenvalues of matrix $\Sigma$, while solution vectors $v$ correspond to the eigenvectors.

### Sparse PCA

Sparse PCA is a special case of the regular PCA, where we require projection vectors to be k-sparse $|v|_0 = k$. Speaking english, it means that only $k$ coordinates of the vector $v$ are allowed to be non-zero, so that we take into account only a subset of columns of matrix $X$ and $\Sigma$ - corresponding to a subset of features that we want to take into account. This alters the requirements:

$\begin{cases} v^T \Sigma v \to max \\ ||v||_2 = 1 \\ ||v||_0 = k \end{cases}$

Unfortunately, this also breaks the useful property of orthogonality of solutions $v_1$, $v_2$ etc.

## Problem 1: minimize Frobenius norm of the difference between data matrix $X$ and its approximation with rank-1 matrix $W$

We are given an input data matrix $X$ of $p$ columns and $n$ rows (e.g. rows are patients and columns are gene expressions). Typically $p >> n$, but not necessarily. E.g.

$X = \begin{pmatrix} 1 && 2 && 3 \\ 4 && 5 && 6 \end{pmatrix}$

We need to find a subset of genes $v = (0, 1, 1, 0, 0, 0)$ and a subset of patients $u = (1, 0, 0, 1)$, such that a subset of those genes and patients is explains most of the variance of matrix X.

This problem is very close to the typical formulation of PCA, but not exactly.

In order to achieve sparcity of vectors $v$ and $u$ (boundedness in $L_0$ norm) we approximate $L_0$ norm with $L_1$ norm in style of Lasso regression.

Concatenate vectors $u$ and $v$ into $w = u \oplus v$ and replace the matrix $X$ with Jordan-Wielandt matrix:

$\bar{X} = \begin{pmatrix} 0 && 0 && 1 && 2 && 3 \\ 0 && 0 && 4 && 5 && 6 \\ 1 && 4 && 0 && 0 && 0 \\ 2 && 5 && 0 && 0 && 0 \\ 3 && 6 && 0 && 0 && 0 \end{pmatrix}$

Now try approximating it with rank-1 matrix $W = w w^T$, which is an outer product of a vector $w$ with itself, with requirement that this vector is sufficiently sparse and its length (L2) norm is limited. Minimize the Forbenius norm of the difference between the real data matrix and this approximation:

$\hat{w} = \arg \min \limits_w || \bar{X} - w w^T ||_F$, given $||w||_1 = c_1$ and $||w||_2 = c_2$.

## Problem 2: customized sparse PCA via Jordan-Wielandt matrix

We are given an input data matrix $X$ of $p$ columns and $n$ rows (e.g. rows are patients and columns are gene expressions). Typically $p >> n$, but not necessarily. E.g.

$X = \begin{pmatrix} 1 && 2 && 3 \\ 4 && 5 && 6 \end{pmatrix}$

We are looking for vectors $u$ and $v$, such that bilinear form $u^T X v$ takes the largest value possible, and $u$ and $v$ vectors are sufficiently sparse (concatenated, have no more than $\alpha$ non-zero entries):

$\begin{cases} u^T X v \to \max \limits_{u, v} \\ ||u \oplus v||_1 \le \alpha \end{cases}$

This problem is easily re-formulated as a sparse PCA problem, if again we do a little transform: $w = u \oplus v$ and replace matrix $X$ with its Jordan-Wieldant matrix:

$\bar{X} = \frac{1}{2} \begin{pmatrix} 0 && X \\ X^T && 0 \end{pmatrix}$

Then our initial problem is equivalent to:

$\begin{cases} w^T \bar{X} w \to \max \limits_{w} \\ ||w||_1 \le \alpha \end{cases}$

This problem is exactly equivalent to SCoTLASS alogrithm for sparse PCA, and can be solved in several ways (coordinate descent as in sklearn Lasso, semidefinite programming, NMF/EM-style structured sparse PCA in scikit-learn).

### Coordinate descent solver.

$\mathcal{L}(w) = w^T \bar{X} w - \alpha ||w||_1$


$sub \frac{\partial L}{\partial w_i} = 2 \sum \limits_{j=1}^{n+p} \bar{X}_{i,j} w_j - \alpha \cdot sub \frac{\partial |w_j|}{\partial w_j} = 0$

Eliminating 2 and $\frac{1}{2}$ and replacing elements of the matrix $\bar{X}_i$ with the elements of X_i, we get:

$\alpha \cdot sub \frac{\partial{|w_j|}}{\partial w_j} = \sum \limits_{j=1}^{n+p} X_{i,j} w_j$

$\alpha \cdot sub \frac{\partial{|w_j|}}{\partial w_j} = \sum \limits_{j \ne i}^{n+p} X_{i,j} w_j + X_{i,i} w_i$

$w_i = \frac{ \alpha \cdot sub \frac{\partial{|w_j|}}{\partial w_j} - \sum \limits_{j \ne i}^{n+p} X_{i,j} w_j }{ X_{i,i} }$

Consider 4 cases now:

1) $\sum \limits_{j \ne i}^{n+p} X_{i,j} w_j > 0$, $|\sum \limits_{j \ne i}^{n+p} X_{i,j} w_j| > \alpha$

If $w_i < 0$ meaning that $sub \frac{\partial |w|}{\partial w} = -1$ and $w_i = \frac{ -\alpha - \sum \limits_{j \ne i}^{n+p} X_{i,j} w_j }{ X_{i,i} }$.

If $w_i = 0$ this leads to a contradiction, because $|sub \frac{\partial |w|}{\partial w}| < 1$.

If $w_i > 0$ this leads to a contradiction, as $|sub \frac{\partial |w|}{\partial w}| < 1$ and $w_i$ has to be negative.

2) $\sum \limits_{j \ne i}^{n+p} X_{i,j} w_j > 0$, $|\sum \limits_{j \ne i}^{n+p} X_{i,j} w_j| < \alpha$

If $w_i < 0$ $sub \frac{\partial |w|}{\partial w} = -1$ and $w_i = \frac{-\alpha - \sum \limits_{j \ne i}^{n+p} X_{i,j} w_j }{ X_{i,i} }$.

If $w_i = 0$ this can be achieved.

If $w_i > 0$ again this can be achieved: $sub \frac{\partial |w|}{\partial w} = -1$ and $w_i = \frac{\alpha - \sum \limits_{j \ne i}^{n+p} X_{i,j} w_j }{ X_{i,i} }$.

3) $\sum \limits_{j \ne i}^{n+p} X_{i,j} w_j < 0$, $|\sum \limits_{j \ne i}^{n+p} X_{i,j} w_j| > \alpha$

If $w_i < 0$ we come to a contradiction.

If $w_i = 0$ this leads to a contradiction.

If $w_i > 0$ $sub \frac{\partial |w|}{\partial w} = 1$ and $w_i = \frac{\alpha - \sum \limits_{j \ne i}^{n+p} X_{i,j} w_j }{ X_{i,i} }$

4) $\sum \limits_{j \ne i}^{n+p} X_{i,j} w_j < 0$, $|\sum \limits_{j \ne i}^{n+p} X_{i,j} w_j| < \alpha$

If $w_i < 0$ $sub \frac{\partial |w|}{\partial w} = -1$ and $w_i = \frac{-\alpha - \sum \limits_{j \ne i}^{n+p} X_{i,j} w_j }{ X_{i,i} }$.

If $w_i = 0$.

If $w_i > 0$ $sub \frac{\partial |w|}{\partial w} = 1$ and $w_i = \frac{\alpha - \sum \limits_{j \ne i}^{n+p} X_{i,j} w_j }{ X_{i,i} }$.

$ X_{i,i}w_i = \begin{cases} \alpha \cdot 1 - \sum \limits_{j=1}^{n+p} X_{i,j} w_j, w_i > 0 \\ \alpha \cdot (-1) - \sum \limits_{j=1}^{n+p} X_{i,j} w_j, w_i < 0 \\ [-\alpha, \alpha] - \sum \limits_{j=1}^{n+p} X_{i,j} w_j, w_i = 0 \end{cases}$


## Equivalence of sparse PCA and Frobenius norm minimization problems

Condier the Frobenius norm minimization problem:

$\hat{w} = \arg \min \limits_{w} ||X - w w^T||_F$

Denote $W = w w^T$ the matrix of rank 1.

Then our problem can be re-written as:

$\hat{w} = \arg \min \limits_{w} ||X - W||_F$

Now re-write the Frobenius norm as a trace of matrix product:

$||X - W||_F = \sqrt{\operatorname{Tr}((X-W)^T(X-W))}$ and $||X - W||_F^2 = \operatorname{Tr}((X-W)^T(X-W)) = \operatorname{Tr}(X^T X - W^T X - X^T W + W^T W)$

$W$ is symmetric. If $X$ is symmetric as well, we get:

$\hat{w} = \arg \min \limits_{w} \operatorname{Tr}(X^T X - W^T X - X^T W + W^T W) = \arg \min \limits_{w} \operatorname{Tr}(X^T X - 2 X^T W + W^T W)$

Now, [trace of a sum of matrices is a sum of their traces](https://proofwiki.org/wiki/Trace_of_Sum_of_Matrices_is_Sum_of_Traces): $\operatorname{Tr}(A + B) = \operatorname{Tr}(A) + \operatorname{Tr}(B)$. Hence:

$\hat{w} = \arg \min \limits_{w} \operatorname{Tr}(X^T X) - 2 \operatorname{Tr}(X^T W) + \operatorname{Tr}(W^T W)$

$\operatorname{Tr}(X^T X)$ is constant, $\operatorname{Tr}(W^T W)$ is constant up to L2 norm (length) of vectors $w$.

Hence

$\hat{w} = \arg \min \limits_{w} ||X - w w^T||_F = \arg \max \limits_{w} \operatorname{Tr}(X^T W)$

Again, using [commutativity of trace under matrix product](https://planetmath.org/proofofpropertiesoftraceofamatrix) ($\operatorname{Tr}(A B) = \operatorname{Tr}(B A)$), we get:

$\hat{w} = \arg \max \limits_{w} \operatorname{Tr}(X^T W) = \arg \max \limits_{w} \operatorname{Tr}(W X^T) = \arg \max \limits_{w} \operatorname{Tr}(W x x^T) = \arg \max \limits_{w} x W x^T$

Thus, we come to the equivalence of problems of minimization of Frobenius norm of matrix approximation by a rank-1 matrix $||W - xx^T||_F$ and maximiation of quadratic form $x W x^T$.

## References
* https://en.wikipedia.org/wiki/Sparse_PCA
* https://arxiv.org/abs/1605.07234 - Bilinear assignment problem (e.g. optimal worker-job pairing)
* http://www.assignmentproblems.com/doc/LSAPIntroduction.pdf - linear sum assignment problem
* https://www.quantiki.org/wiki/trace-norm - on trace norm
* https://arxiv.org/pdf/1109.1990.pdf - trace Lasso, Jordan-Wielandt matrix
* https://arxiv.org/pdf/1608.04517.pdf - nuclear norm minimization methods, WNNM
* https://escholarship.org/content/qt33k509ss/qt33k509ss_noSplash_2977f5010d5a3433a7faad29576f1a41.pdf - masters thesis on trace norm
* https://cs.uwaterloo.ca/~yboykov/miccai14_MIS/daniel_variational_methods9.pdf - convex relaxation methods
* https://davidrosenberg.github.io/mlcourse/Notes/convex-optimization.pdf - extreme abridgement of Boyd-Vanderberghe
* https://proceedings.neurips.cc/paper/2006/file/0afa92fc0f8a9cf051bf2961b06ac56b-Paper.pdf - multitask feature learning
* https://hastie.su.domains/Papers/spc_jcgs.pdf - original sparse PCA paper by Zou, Hastie, Tibs
* https://www.semanticscholar.org/paper/Principal-component-analysis%3A-a-review-and-recent-Jolliffe-Cadima/5bc875d65df812f9617d8ba508c1c85f4d219b19 - Cadima-Jolliffe predecessor paper to sPCA
* https://www.researchgate.net/publication/42790702_A_Modified_Principal_Component_Technique_Based_on_the_LASSO - Jolliffe, Trendafilov, Udin original paper on SCoTLASS
* https://www.semanticscholar.org/paper/Feature-Extraction-by-the-SCoTLASS%3A-An-Illustrative-Bartkowiak-Trendafilov/0d4c8a97a805797d5d4183c55283734321a424df - SCoTLASS example paper by Trendafilov
* https://scikit-learn.org/dev/modules/decomposition.html - sparse PCA implementation in scikit-learn
* https://www.di.ens.fr/~fbach/sspca_AISTATS2010.pdf - reference algorithm for sparse PCA implementation in scikit-learn, structured sparse PCA (ssPCA) 
* https://www.di.ens.fr/sierra/pdfs/icml09.pdf - another reference algorithm for sparse PCA in scikit-learn
* https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2697346/ - Witten, Hastie, Tibs on penalized matrix decomposition for sparse PCA and CCA
* https://arxiv.org/pdf/0904.3523.pdf - pre-requisite paper to reference sparse PCA implementation (structured sparse PCA)
* https://www.arpapress.com/Volumes/Vol6Issue4/IJRRAS_6_4_02.pdf - Jordan-Wielandt matrix and Weyl theorem
* https://www.sciencedirect.com/science/article/pii/S0024379502007334?ref=pdf_download&fr=RR-2&rr=7193df34ec9e9007 - on Wielandt and Ky Fan theorems
* https://math.stackexchange.com/questions/1077629/perturbation-theory-of-the-eigenvalues-about-the-symmetric-matrix - Weyl theorem
* http://eceweb.ucsd.edu/~gert/papers/sparse_pca_nips.pdf - El Ghaoui, Michael I. Jordan implementation of sPCA via semidefinite programming
* https://cran.r-project.org/web/packages/rospca/rospca.pdf - robust sparse PCA package
* https://arxiv.org/pdf/0912.3599.pdf - robust PCA paper by E.Candes